In [ ]:
# this code generated videos for each synthesized trajectories

import os
import random
import pickle
import cv2
import numpy as np
import h5py
import torch
from scipy.spatial.transform import Rotation as R
from collections import defaultdict
from PIL import Image
import plotly.graph_objects as go
from carvekit.api.high import HiInterface
from vivid123.generation_utils import generation_vivid123_new, prepare_vivid123_pipeline

###############################################################################
# 1) Helper Functions for Trajectory Generation
###############################################################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

def make_cube(cx, cy, cz, rotation, side=0.005):
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    return rotated_corners

def generate_image_frame(img_width, img_height, point_coord, dot_dia=10):
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    cv2.circle(image, point_coord, dot_dia//2, (255, 255, 255), -1)
    return image

def generate_camera_image(camera_pose, camera_fov, point_obj, 
                          img_width=640, img_height=480, dot_dia=10):
    cam_pos = np.array(camera_pose[:3])
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    img_x = int((h_angle + h_fov/2) / h_fov * img_width)
    img_y = int((v_angle + v_fov/2) / v_fov * img_height)
    return generate_image_frame(img_width, img_height, (img_x, img_y), dot_dia)

def is_inFOV(camera_pose, camera_fov, point_obj):
    cam_pos = np.array(camera_pose[:3])
    if np.linalg.norm(np.array(point_obj) - cam_pos) < 1e-4:
        return True
    cam_quat = camera_pose[3:]
    cam_rot = R.from_quat(cam_quat).as_matrix()
    point_in_cam = np.linalg.inv(cam_rot) @ (np.array(point_obj) - cam_pos)
    h_fov, v_fov = np.radians(camera_fov)
    x_cam, y_cam, z_cam = point_in_cam
    if x_cam <= 0:
        return False
    h_angle = np.arctan2(y_cam, x_cam)
    v_angle = np.arctan2(z_cam, x_cam)
    return (abs(h_angle) <= h_fov/2) and (abs(v_angle) <= v_fov/2)

def unit_vector(v):
    norm = np.linalg.norm(v)
    if norm < 1e-9:
        return np.array([1, 0, 0])
    return v / norm

def regenerate_candidates(i, base_positions, base_rpy, num_new_samples, reduction_factor,
                          goal_pos, lambda_decay, deg_to_rad, max_deg, max_side,
                          max_roll, max_pitch, max_yaw):
    candidates = []
    x, y, z = base_positions[i]
    roll_i, pitch_i, yaw_i = base_rpy[i]
    dx = x - goal_pos[0]
    dy = y - goal_pos[1]
    dz = z - goal_pos[2]
    r_val, az, el = cart2sph(dx, dy, dz)
    for sample in range(num_new_samples):
        frac = np.exp(-lambda_decay * (i / (len(base_positions) - 1))) if len(base_positions)>1 else 1.0
        delta_angle = frac * (max_deg * deg_to_rad) * reduction_factor
        az_var = az + random.choice([-1, 1]) * delta_angle
        el_var = el + random.choice([-1, 1]) * delta_angle
        dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
        x_new = goal_pos[0] + dx_var
        y_new = goal_pos[1] + dy_var
        z_new = goal_pos[2] + dz_var

        roll_off  = frac * max_roll * reduction_factor
        pitch_off = frac * max_pitch * reduction_factor
        yaw_off   = frac * max_yaw * reduction_factor
        droll = roll_i - base_rpy[-1][0]
        dpitch = pitch_i - base_rpy[-1][1]
        dyaw = yaw_i - base_rpy[-1][2]
        droll_var = droll + random.choice([-1, 1]) * roll_off
        dpitch_var = dpitch + random.choice([-1, 1]) * pitch_off
        dyaw_var = dyaw + random.choice([-1, 1]) * yaw_off
        roll_new = base_rpy[-1][0] + droll_var
        pitch_new = base_rpy[-1][1] + dpitch_var
        yaw_new = base_rpy[-1][2] + dyaw_var

        rot_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
        side_len = frac * max_side
        if side_len < 1e-9:
            continue
        corners = make_cube(x_new, y_new, z_new, rot_obj, side_len)
        for j, corner in enumerate(corners):
            candidates.append({
                "trajectory_idx": i,
                "pose_6d": [corner[0], corner[1], corner[2],
                            roll_new, pitch_new, yaw_new],
                "in_fov": True  # assume resampled ones are likely in FoV
            })
    return candidates

###############################################################################
# 2) Trajectory Generation Function
###############################################################################

def iterative_trajectory_generation():
    # --- A) Load main trajectory from HDF5
    file_path = "demo_duck_feb12.hdf5"  # Adjust path if needed.
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        # Use the first demo for trajectory generation.
        demo_name = demo_keys[0]
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            states = obs_group["joint_states"][:]  # shape: (T,7)
            positions_list.append(states[:, :3])
            orientations_list.append(states[:, 3:6])
    if not positions_list:
        print("No trajectory data found.")
        return None

    positions = positions_list[0]  # (N,3)
    rpy_list = orientations_list[0]  # (N,3)
    n_points = len(positions)
    if n_points < 2:
        print("Not enough main trajectory points.")
        return None

    # Precompute main trajectory's local directions.
    main_dirs = []
    for i in range(1, n_points):
        diff = positions[i] - positions[i-1]
        norm = np.linalg.norm(diff)
        if norm < 1e-9:
            main_dirs.append(np.array([1, 0, 0]))
        else:
            main_dirs.append(diff / norm)
    main_dirs.insert(0, main_dirs[0])

    # --- B) Base variation parameters.
    base_max_deg = 5.0
    base_max_side = 0.1
    base_max_roll = 50 * np.pi/180.0
    base_max_pitch = 10 * np.pi/180.0
    base_max_yaw = 10 * np.pi/180.0

    lambda_decay = 1.5
    camera_fov = (80, 80)
    img_width, img_height, dot_dia = 640, 480, 10
    num_samples = 10

    goal_pos = positions[-1]
    goal_rpy = rpy_list[-1]

    # --- C) Compute Global Reference Spherical Coordinates.
    dx_ref = positions[0][0] - goal_pos[0]
    dy_ref = positions[0][1] - goal_pos[1]
    dz_ref = positions[0][2] - goal_pos[2]
    ref_r, ref_az, ref_el = cart2sph(dx_ref, dy_ref, dz_ref)
    print(f"Reference spherical coords: r={ref_r:.3f}, az={ref_az:.3f}, el={ref_el:.3f}")

    trajectories = []
    iter_idx = 0
    variation_scale = 0.1  # start with very little variation

    # Stop if candidate FoV ratio becomes too low.
    min_fov_ratio = 0.5

    while True:
        max_deg = base_max_deg * variation_scale
        max_side = base_max_side * variation_scale
        max_roll = base_max_roll * variation_scale
        max_pitch = base_max_pitch * variation_scale
        max_yaw = base_max_yaw * variation_scale

        print(f"\nIteration {iter_idx}, variation scale {variation_scale:.3f}")
        dataset = []
        total_candidates = 0
        in_fov_candidates = 0

        for i in range(n_points):
            x, y, z = positions[i]
            roll_i, pitch_i, yaw_i = rpy_list[i]
            dx = x - goal_pos[0]
            dy = y - goal_pos[1]
            dz = z - goal_pos[2]
            r_val, az, el = cart2sph(dx, dy, dz)
            droll = roll_i - goal_rpy[0]
            dpitch = pitch_i - goal_rpy[1]
            dyaw = yaw_i - goal_rpy[2]
            for sample in range(num_samples):
                frac = np.exp(-lambda_decay * (i / (n_points - 1))) if n_points > 1 else 1.0
                delta_angle = frac * (max_deg * np.pi/180.0)
                az_var = az + random.choice([-1, 1]) * delta_angle
                el_var = el + random.choice([-1, 1]) * delta_angle
                dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
                x_new = goal_pos[0] + dx_var
                y_new = goal_pos[1] + dy_var
                z_new = goal_pos[2] + dz_var

                roll_off = frac * max_roll
                pitch_off = frac * max_pitch
                yaw_off = frac * max_yaw
                droll_var = droll + random.choice([-1, 1]) * roll_off
                dpitch_var = dpitch + random.choice([-1, 1]) * pitch_off
                dyaw_var = dyaw + random.choice([-1, 1]) * yaw_off
                roll_new = goal_rpy[0] + droll_var
                pitch_new = goal_rpy[1] + dpitch_var
                yaw_new = goal_rpy[2] + dyaw_var

                rot_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
                side_len = frac * max_side
                if side_len < 1e-9:
                    continue
                corners = make_cube(x_new, y_new, z_new, rot_obj, side_len)
                # Compute candidate center's spherical coordinates.
                r_center, az_center, el_center = cart2sph(x_new - goal_pos[0],
                                                          y_new - goal_pos[1],
                                                          z_new - goal_pos[2])
                # Delta relative to the global reference.
                delta_az = az_center - ref_az
                delta_el = el_center - ref_el
                delta_r = r_center - ref_r

                quat = rot_obj.as_quat()
                for j, corner in enumerate(corners):
                    total_candidates += 1
                    cam_pose = (corner[0], corner[1], corner[2],
                                quat[0], quat[1], quat[2], quat[3])
                    in_fov = is_inFOV(cam_pose, camera_fov, goal_pos)
                    if in_fov:
                        in_fov_candidates += 1
                    dataset.append({
                        "trajectory_idx": i,
                        "pose_6d": [corner[0], corner[1], corner[2],
                                    roll_new, pitch_new, yaw_new],
                        "delta_pose": [delta_az, delta_el, delta_r],
                        "in_fov": in_fov,
                        "image": generate_camera_image(cam_pose, camera_fov, goal_pos,
                                                       img_width, img_height, dot_dia)
                    })
        ratio = in_fov_candidates / total_candidates if total_candidates > 0 else 0
        print(f"Candidates: {total_candidates}, InFoV: {in_fov_candidates} (ratio {ratio:.2f})")
        if ratio < min_fov_ratio:
            print("FoV ratio too low. Stopping iterations.")
            break

        # Build a trajectory using a greedy chain.
        in_fov_dataset = [vp for vp in dataset if vp["in_fov"]]
        in_fov_dataset = sorted(in_fov_dataset, key=lambda vp: vp["trajectory_idx"])
        first_group = [vp for vp in in_fov_dataset if vp["trajectory_idx"] == 0]
        if not first_group:
            print("No candidates in the first group. Aborting iteration.")
            break
        current = min(first_group, key=lambda vp: np.linalg.norm(np.array(vp["pose_6d"][:3]) - positions[0]))
        chain = [current]
        current_pos = np.array(current["pose_6d"][:3])
        for i in range(1, n_points):
            group = [vp for vp in in_fov_dataset if vp["trajectory_idx"] == i]
            if not group:
                continue
            desired_dir = unit_vector(positions[i] - positions[i-1])
            best_candidate = None
            best_cost = float('inf')
            for cand in group:
                cand_pos = np.array(cand["pose_6d"][:3])
                diff = cand_pos - current_pos
                dist = np.linalg.norm(diff)
                if dist < 1e-6:
                    continue
                cand_dir = unit_vector(diff)
                angle = np.arccos(np.clip(np.dot(cand_dir, desired_dir), -1.0, 1.0))
                cost = dist + 10.0 * angle
                if cost < best_cost:
                    best_cost = cost
                    best_candidate = cand
            max_attempts = 5
            attempt = 0
            while best_candidate is None and attempt < max_attempts:
                print(f"Resampling at index {i}, attempt {attempt+1}")
                new_cands = regenerate_candidates(i, positions, rpy_list, num_new_samples=5,
                                                  reduction_factor=0.5, goal_pos=goal_pos,
                                                  lambda_decay=lambda_decay, deg_to_rad=np.pi/180.0,
                                                  max_deg=base_max_deg, max_side=base_max_side,
                                                  max_roll=base_max_roll, max_pitch=base_max_pitch,
                                                  max_yaw=base_max_yaw)
                for cand in new_cands:
                    dataset.append(cand)
                    cand_pos = np.array(cand["pose_6d"][:3])
                    diff = cand_pos - current_pos
                    dist = np.linalg.norm(diff)
                    if dist < 1e-6:
                        continue
                    cand_dir = unit_vector(diff)
                    angle = np.arccos(np.clip(np.dot(cand_dir, desired_dir), -1.0, 1.0))
                    cost = dist + 10.0 * angle
                    if cost < best_cost:
                        best_cost = cost
                        best_candidate = cand
                attempt += 1
            if best_candidate is None:
                print(f"Skipping index {i} due to no good candidate.")
                continue
            chain.append(best_candidate)
            current_pos = np.array(best_candidate["pose_6d"][:3])
        print(f"Iteration {iter_idx}: Generated chain with {len(chain)} points.")
        trajectories.append(chain)
        variation_scale *= 1.2
        iter_idx += 1
        if iter_idx >= 10:
            break

    # (Optional) Visualize the main trajectory and generated chains.
    # fig = go.Figure()
    # main_x = [p[0] for p in positions]
    # main_y = [p[1] for p in positions]
    # main_z = [p[2] for p in positions]
    # fig.add_trace(go.Scatter3d(
    #     x=main_x, y=main_y, z=main_z,
    #     mode='lines+markers',
    #     line=dict(color='red', width=4),
    #     name='Main Trajectory'
    # ))
    # color_list = ["green", "blue", "orange", "purple", "brown", "cyan", "magenta", "yellow", "pink", "gray"]
    # for idx, chain in enumerate(trajectories):
    #     chain_pts = [np.array(vp["pose_6d"][:3]) for vp in chain]
    #     if not chain_pts:
    #         continue
    #     chain_x = [pt[0] for pt in chain_pts]
    #     chain_y = [pt[1] for pt in chain_pts]
    #     chain_z = [pt[2] for pt in chain_pts]
    #     col = color_list[idx % len(color_list)]
    #     fig.add_trace(go.Scatter3d(
    #         x=chain_x, y=chain_y, z=chain_z,
    #         mode='lines+markers',
    #         line=dict(color=col, width=3),
    #         marker=dict(size=3),
    #         name=f'Trajectory {idx+1}'
    #     ))
    # fig.update_layout(
    #     title="Main Trajectory (Red) and Generated Trajectories",
    #     scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
    #     margin=dict(l=0, r=0, b=0, t=40)
    # )
    # fig.show()
    return trajectories

###############################################################################
# 3) Synthesis & Video Generation
###############################################################################

def synthesize_and_create_videos(trajectories):
    # ---------------------------
    # A) Load Reference Image from HDF5 and Remove Background
    # ---------------------------
    hdf5_file_path = "demo_duck_feb12.hdf5"  # Same file used for trajectory generation.
    demo_key = list(h5py.File(hdf5_file_path, "r")["data"].keys())[0]
    with h5py.File(hdf5_file_path, "r") as f:
        # Assume the reference image is stored under "obs"/"eye_in_hand_rgb"
        ref_image_data = np.array(f["data"][demo_key]["obs"]["eye_in_hand_rgb"][0])
    if ref_image_data.shape[-1] == 3:
        ref_image_data = ref_image_data[..., ::-1]  # Convert BGR->RGB if needed.
    ref_image = Image.fromarray(ref_image_data)
    temp_ref_folder = "temp_ref"
    os.makedirs(temp_ref_folder, exist_ok=True)
    ref_image_path = os.path.join(temp_ref_folder, "eye_in_hand_rgb.png")
    ref_image.save(ref_image_path)
    
    # Initialize background removal interface.
    bg_removal_interface = HiInterface(
        object_type="hairs-like",
        batch_size_seg=5,
        batch_size_matting=1,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        seg_mask_size=640,
        matting_mask_size=2048,
        trimap_prob_threshold=231,
        trimap_dilation=30,
        trimap_erosion_iters=5,
        fp16=False
    )
    print("Removing background from reference image...")
    processed_images = bg_removal_interface([ref_image_path])
    processed_ref_image = processed_images[0]
    processed_ref_image.save(ref_image_path)
    print(f"Processed reference image saved to: {ref_image_path}")

    # ---------------------------
    # B) Setup vivid123 Pipeline and Config
    # ---------------------------
    ZERO123_MODEL_ID = "bennyguo/zero123-xl-diffusers"
    VIDEO_MODEL_ID = "cerspense/zeroscope_v2_576w"
    VIDEO_XL_MODEL_ID = "cerspense/zeroscope_v2_XL"

    vivid123_pipe, xl_pipe = prepare_vivid123_pipeline(
        ZERO123_MODEL_ID=ZERO123_MODEL_ID, 
        VIDEO_MODEL_ID=VIDEO_MODEL_ID, 
        VIDEO_XL_MODEL_ID=VIDEO_XL_MODEL_ID
    )

    # Include "generation_type" in the config.
    config = {
        "delta_azimuth_end": 0.0,
        "delta_azimuth_start": 0.0,
        "delta_elevation_end": 0.0,
        "delta_elevation_start": 0.0,
        "delta_radius_end": 0.0,
        "delta_radius_start": 0.0,
        "eta": 0.5,
        "guidance_scale_video": 2.0,
        "guidance_scale_zero123": 6.0,
        "height": 256,
        "width": 256,
        "input_image_path": ref_image_path,  # Use the background-removed reference image.
        "obj_name": "duck",  # Will update per synthesized output.
        "noise_identical_accross_frames": False,
        "num_frames": 1,
        "num_inference_steps": 50,
        "prompt": "a toy duck",
        "refiner_guidance_scale": 1.0,
        "refiner_strength": 0.1,
        "video_end_step_percentage": 1.0,
        "video_linear_end_weight": 0.5,
        "video_linear_start_weight": 1.0,
        "video_start_step_percentage": 0.0,
        "zero123_end_step_percentage": 1.0,
        "zero123_linear_end_weight": 1.0,
        "zero123_linear_start_weight": 1.0,
        "zero123_start_step_percentage": 0.0,
        "generation_type": "image"  # Required key.
    }

    # ---------------------------
    # C) Synthesize Images for Each Candidate and Collect Image Paths
    # ---------------------------
    synth_output_folder = "vivid123_synthesized_images"
    os.makedirs(synth_output_folder, exist_ok=True)
    trajectory_image_paths = []  # List of lists.

    for traj_idx, traj in enumerate(trajectories):
        print(f"\nSynthesizing images for trajectory {traj_idx} with {len(traj)} points...")
        traj_image_paths = []
        for point_idx, point in enumerate(traj):
            # Get delta_pose (expected [delta_az, delta_el, delta_r]).
            delta_pose = point.get("delta_pose", [0.0, 0.0, 0.0])
            delta_az, delta_el, delta_r = delta_pose

            # Update config with the delta values.
            config["delta_azimuth_start"] = delta_az
            config["delta_azimuth_end"]   = delta_az
            config["delta_elevation_start"] = delta_el
            config["delta_elevation_end"]   = delta_el
            config["delta_radius_start"]    = delta_r
            config["delta_radius_end"]      = delta_r

            # Instead of setting obj_name to a file path with .png extension,
            # create a directory for each synthesized output.
            output_dir = os.path.join(synth_output_folder, f"traj_{traj_idx}_point_{point_idx}")
            os.makedirs(output_dir, exist_ok=True)
            config["obj_name"] = output_dir

            print(f"Trajectory {traj_idx}, point {point_idx}: Δaz={delta_az:.3f}, Δel={delta_el:.3f}, Δr={delta_r:.3f}")
            try:
                generation_vivid123_new(config=config, vivid123_pipe=vivid123_pipe, xl_pipe=xl_pipe)
                # Assume the synthesized image is saved as 'image000.png' inside the output_dir.
                synth_img_path = os.path.join(output_dir, "image000.png")
                if os.path.exists(synth_img_path):
                    traj_image_paths.append(synth_img_path)
                else:
                    print(f"Expected synthesized image not found in {output_dir}.")
            except Exception as e:
                print(f"Error synthesizing image for traj {traj_idx}, point {point_idx}: {e}")
        trajectory_image_paths.append(traj_image_paths)

    # ---------------------------
    # D) Generate Video for Each Trajectory using Synthesized Images
    # ---------------------------
    video_output_folder = "vivid123_videos"
    os.makedirs(video_output_folder, exist_ok=True)
    fps = 2  # Adjust FPS as desired.
    frame_size = (config["width"], config["height"])

    for traj_idx, img_paths in enumerate(trajectory_image_paths):
        if not img_paths:
            print(f"Trajectory {traj_idx} has no synthesized images; skipping video creation.")
            continue

        video_filename = os.path.join(video_output_folder, f"trajectory_{traj_idx}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(video_filename, fourcc, fps, frame_size)
        print(f"Creating video for trajectory {traj_idx} with {len(img_paths)} frames...")
        for img_path in img_paths:
            if not os.path.exists(img_path):
                print(f"Image {img_path} not found; skipping.")
                continue
            frame = cv2.imread(img_path)
            if frame is None:
                print(f"Failed to read {img_path}; skipping.")
                continue
            frame = cv2.resize(frame, frame_size)
            video_writer.write(frame)
        video_writer.release()
        print(f"Video saved to: {video_filename}")

    print("Synthesis and video generation completed.")
    return trajectory_image_paths

###############################################################################
# 4) Main Function
###############################################################################

def main():
    # First, generate trajectories from the demo HDF5.
    trajectories = iterative_trajectory_generation()
    if trajectories is None:
        print("No trajectories generated. Exiting.")
        return

    # Then, synthesize camera view images for each trajectory and create videos.
    synthesize_and_create_videos(trajectories)

if __name__ == "__main__":
    main()


Available Demos: ['demo_0', 'demo_1', 'demo_2', 'demo_3', 'demo_4', 'demo_5']
Reference spherical coords: r=0.717, az=-1.481, el=0.304

Iteration 0, variation scale 0.100
Candidates: 22720, InFoV: 21125 (ratio 0.93)
Iteration 0: Generated chain with 282 points.

Iteration 1, variation scale 0.120
Candidates: 22720, InFoV: 21004 (ratio 0.92)
Iteration 1: Generated chain with 282 points.

Iteration 2, variation scale 0.144
Candidates: 22720, InFoV: 20827 (ratio 0.92)
Iteration 2: Generated chain with 282 points.

Iteration 3, variation scale 0.173
Candidates: 22720, InFoV: 20682 (ratio 0.91)
Iteration 3: Generated chain with 282 points.

Iteration 4, variation scale 0.207
Candidates: 22720, InFoV: 20456 (ratio 0.90)
Iteration 4: Generated chain with 281 points.

Iteration 5, variation scale 0.249
Candidates: 22720, InFoV: 19817 (ratio 0.87)
Iteration 5: Generated chain with 280 points.

Iteration 6, variation scale 0.299
Candidates: 22720, InFoV: 19234 (ratio 0.85)
Iteration 6: Generated

text_encoder/model.safetensors not found
Loading pipeline components...: 100%|██████████| 5/5 [00:02<00:00,  1.84it/s]
text_encoder/model.safetensors not found
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  5.00it/s]



Synthesizing images for trajectory 0 with 282 points...
Trajectory 0, point 0: Δaz=0.009, Δel=-0.009, Δr=-0.000
input_image_path is: temp_ref/eye_in_hand_rgb.png


100%|██████████| 5/5 [00:00<00:00,  5.33it/s]


Saving output images.....
Trajectory 0, point 1: Δaz=0.009, Δel=-0.008, Δr=-0.001
input_image_path is: temp_ref/eye_in_hand_rgb.png


...

Saving output images.....
Trajectory 0, point 13: Δaz=-0.011, Δel=-0.012, Δr=0.009
input_image_path is: temp_ref/eye_in_hand_rgb.png


100%|██████████| 5/5 [00:00<00:00,  6.26it/s]


Saving output images.....
Trajectory 0, point 14: Δaz=-0.011, Δel=-0.012, Δr=0.008
input_image_path is: temp_ref/eye_in_hand_rgb.png


  0%|          | 0/50 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [4]:
import os
import random
import cv2
import numpy as np
import h5py
import torch
from scipy.spatial.transform import Rotation as R
from PIL import Image
from carvekit.api.high import HiInterface
from vivid123.generation_utils import generation_vivid123_new, prepare_vivid123_pipeline

#########################################
# Helper Functions for Trajectory Generation
#########################################

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    az = np.arctan2(y, x)
    el = np.arctan2(z, np.sqrt(x**2 + y**2))
    return r, az, el

def sph2cart(r, az, el):
    x = r * np.cos(el) * np.cos(az)
    y = r * np.cos(el) * np.sin(az)
    z = r * np.sin(el)
    return x, y, z

def make_cube(cx, cy, cz, rotation, side=0.005):
    half = side / 2.0
    corners = np.array([
        [-half, -half, -half],
        [-half, -half,  half],
        [-half,  half, -half],
        [-half,  half,  half],
        [ half, -half, -half],
        [ half, -half,  half],
        [ half,  half, -half],
        [ half,  half,  half]
    ])
    rotated_corners = rotation.apply(corners)
    rotated_corners += np.array([cx, cy, cz])
    return rotated_corners

def unit_vector(v):
    norm = np.linalg.norm(v)
    if norm < 1e-9:
        return np.array([1, 0, 0])
    return v / norm

#########################################
# Trajectory Generation Function
#########################################

def iterative_trajectory_generation():
    # Load main trajectory from HDF5 demo
    file_path = "demo_duck_feb12.hdf5"  # Adjust path if needed.
    positions_list = []
    orientations_list = []
    with h5py.File(file_path, "r") as f:
        demo_keys = list(f["data"].keys())
        print("Available Demos:", demo_keys)
        demo_name = demo_keys[0]  # use the first demo
        demo_data = f["data"][demo_name]
        obs_group = demo_data["obs"]
        if "joint_states" in obs_group:
            states = obs_group["joint_states"][:]  # shape: (T,7)
            positions_list.append(states[:, :3])
            orientations_list.append(states[:, 3:6])
    if not positions_list:
        print("No trajectory data found.")
        return None

    positions = positions_list[0]  # (N,3)
    rpy_list = orientations_list[0]  # (N,3)
    n_points = len(positions)
    if n_points < 2:
        print("Not enough main trajectory points.")
        return None

    # Compute global reference (from first main point to goal)
    goal_pos = positions[-1]
    dx_ref = positions[0][0] - goal_pos[0]
    dy_ref = positions[0][1] - goal_pos[1]
    dz_ref = positions[0][2] - goal_pos[2]
    ref_r, ref_az, ref_el = cart2sph(dx_ref, dy_ref, dz_ref)
    print(f"Reference spherical coords: r={ref_r:.3f}, az={ref_az:.3f}, el={ref_el:.3f}")

    # Variation parameters
    base_max_deg = 5.0
    base_max_side = 0.1
    base_max_roll = 50 * np.pi/180.0
    base_max_pitch = 10 * np.pi/180.0
    base_max_yaw = 10 * np.pi/180.0
    lambda_decay = 1.5
    num_samples = 10

    trajectories = []
    variation_scale = 0.1  # starting variation

    # We'll generate one candidate chain (trajectory) using a greedy method.
    dataset = []
    for i in range(n_points):
        x, y, z = positions[i]
        roll_i, pitch_i, yaw_i = rpy_list[i]
        dx = x - goal_pos[0]
        dy = y - goal_pos[1]
        dz = z - goal_pos[2]
        r_val, az, el = cart2sph(dx, dy, dz)
        droll = roll_i - rpy_list[-1][0]
        dpitch = pitch_i - rpy_list[-1][1]
        dyaw = yaw_i - rpy_list[-1][2]
        for sample in range(num_samples):
            frac = np.exp(-lambda_decay * (i / (n_points - 1))) if n_points > 1 else 1.0
            delta_angle = frac * (base_max_deg * np.pi/180.0)
            az_var = az + random.choice([-1, 1]) * delta_angle
            el_var = el + random.choice([-1, 1]) * delta_angle
            dx_var, dy_var, dz_var = sph2cart(r_val, az_var, el_var)
            x_new = goal_pos[0] + dx_var
            y_new = goal_pos[1] + dy_var
            z_new = goal_pos[2] + dz_var

            roll_off = frac * base_max_roll
            pitch_off = frac * base_max_pitch
            yaw_off = frac * base_max_yaw
            droll_var = droll + random.choice([-1, 1]) * roll_off
            dpitch_var = dpitch + random.choice([-1, 1]) * pitch_off
            dyaw_var = dyaw + random.choice([-1, 1]) * yaw_off
            roll_new = rpy_list[-1][0] + droll_var
            pitch_new = rpy_list[-1][1] + dpitch_var
            yaw_new = rpy_list[-1][2] + dyaw_var

            rot_obj = R.from_euler('zyx', [roll_new, pitch_new, yaw_new])
            side_len = frac * base_max_side
            if side_len < 1e-9:
                continue
            corners = make_cube(x_new, y_new, z_new, rot_obj, side_len)
            # Compute the candidate center's spherical coordinates
            r_center, az_center, el_center = cart2sph(x_new - goal_pos[0],
                                                      y_new - goal_pos[1],
                                                      z_new - goal_pos[2])
            delta_az = az_center - ref_az
            delta_el = el_center - ref_el
            delta_r  = r_center - ref_r
            quat = rot_obj.as_quat()
            for corner in corners:
                dataset.append({
                    "trajectory_idx": i,
                    "pose_6d": [corner[0], corner[1], corner[2],
                                roll_new, pitch_new, yaw_new],
                    "delta_pose": [delta_az, delta_el, delta_r],
                    "in_fov": True  # for simplicity, assume all are in view
                })
    # For this test, simply choose a greedy chain by taking the candidate with minimal index per point.
    chain = []
    for i in range(n_points):
        group = [vp for vp in dataset if vp["trajectory_idx"] == i and vp["in_fov"]]
        if group:
            chain.append(group[0])
    trajectories.append(chain)
    print(f"Generated one trajectory with {len(chain)} points.")
    return trajectories

#########################################
# Synthesis & Video Generation for a Single Trajectory
#########################################

def synthesize_and_create_video_for_trajectory(traj):
    # ---- A) Load Reference Image from HDF5 & Remove Background ----
    hdf5_file_path = "demo_duck_feb12.hdf5"  # same file used above
    with h5py.File(hdf5_file_path, "r") as f:
        demo_key = list(f["data"].keys())[0]
        ref_image_data = np.array(f["data"][demo_key]["obs"]["eye_in_hand_rgb"][0])
    if ref_image_data.shape[-1] == 3:
        ref_image_data = ref_image_data[..., ::-1]  # convert BGR->RGB if needed
    ref_image = Image.fromarray(ref_image_data)
    temp_ref_folder = "temp_ref"
    os.makedirs(temp_ref_folder, exist_ok=True)
    ref_image_path = os.path.join(temp_ref_folder, "eye_in_hand_rgb.png")
    ref_image.save(ref_image_path)
    # Background removal using CarveKit
    bg_interface = HiInterface(
        object_type="hairs-like",
        batch_size_seg=5,
        batch_size_matting=1,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        seg_mask_size=640,
        matting_mask_size=2048,
        trimap_prob_threshold=231,
        trimap_dilation=30,
        trimap_erosion_iters=5,
        fp16=False
    )
    print("Removing background from reference image...")
    processed = bg_interface([ref_image_path])
    processed[0].save(ref_image_path)
    print(f"Processed reference image saved to: {ref_image_path}")

    # ---- B) Setup vivid123 Pipeline & Config ----
    ZERO123_MODEL_ID = "bennyguo/zero123-xl-diffusers"
    VIDEO_MODEL_ID = "cerspense/zeroscope_v2_576w"
    VIDEO_XL_MODEL_ID = "cerspense/zeroscope_v2_XL"
    vivid123_pipe, xl_pipe = prepare_vivid123_pipeline(
        ZERO123_MODEL_ID=ZERO123_MODEL_ID, 
        VIDEO_MODEL_ID=VIDEO_MODEL_ID, 
        VIDEO_XL_MODEL_ID=VIDEO_XL_MODEL_ID
    )
    # Config for vivid123; note "generation_type" is required.
    config = {
        "delta_azimuth_end": 0.0,
        "delta_azimuth_start": 0.0,
        "delta_elevation_end": 0.0,
        "delta_elevation_start": 0.0,
        "delta_radius_end": 0.0,
        "delta_radius_start": 0.0,
        "eta": 0.5,
        "guidance_scale_video": 2.0,
        "guidance_scale_zero123": 6.0,
        "height": 256,
        "width": 256,
        "input_image_path": ref_image_path,
        "obj_name": "dummy",  # will be updated per candidate
        "noise_identical_accross_frames": False,
        "num_frames": 1,
        "num_inference_steps": 50,
        "prompt": "a toy duck",
        "refiner_guidance_scale": 1.0,
        "refiner_strength": 0.1,
        "video_end_step_percentage": 1.0,
        "video_linear_end_weight": 0.5,
        "video_linear_start_weight": 1.0,
        "video_start_step_percentage": 0.0,
        "zero123_end_step_percentage": 1.0,
        "zero123_linear_end_weight": 1.0,
        "zero123_linear_start_weight": 1.0,
        "zero123_start_step_percentage": 0.0,
        "generation_type": "image"
    }

    # ---- C) Synthesize Images for Each Point in the Trajectory ----
    synth_folder = "vivid123_synthesized_images_single"
    os.makedirs(synth_folder, exist_ok=True)
    synthesized_paths = []
    for idx, point in enumerate(traj):
        delta_pose = point.get("delta_pose", [0.0, 0.0, 0.0])
        delta_az, delta_el, delta_r = delta_pose
        # Update config with candidate's delta values.
        config["delta_azimuth_start"] = delta_az
        config["delta_azimuth_end"]   = delta_az
        config["delta_elevation_start"] = delta_el
        config["delta_elevation_end"]   = delta_el
        config["delta_radius_start"]    = delta_r
        config["delta_radius_end"]      = delta_r

        # Create a unique output directory for this candidate.
        out_dir = os.path.join(synth_folder, f"point_{idx}")
        os.makedirs(out_dir, exist_ok=True)
        config["obj_name"] = out_dir

        print(f"Processing point {idx}: Δaz={delta_az:.3f}, Δel={delta_el:.3f}, Δr={delta_r:.3f}")
        try:
            generation_vivid123_new(config=config, vivid123_pipe=vivid123_pipe, xl_pipe=xl_pipe)
            img_path = os.path.join(out_dir, "image000.png")
            if os.path.exists(img_path):
                synthesized_paths.append(img_path)
            else:
                print(f"Image not found in {out_dir}.")
        except Exception as e:
            print(f"Error synthesizing point {idx}: {e}")

    # ---- D) Create Video from Synthesized Images ----
    if not synthesized_paths:
        print("No synthesized images available; cannot create video.")
        return

    video_out = "trajectory_video.mp4"
    fps = 2
    frame_size = (config["width"], config["height"])
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(video_out, fourcc, fps, frame_size)
    print(f"Creating video from {len(synthesized_paths)} frames...")
    for sp in synthesized_paths:
        frame = cv2.imread(sp)
        if frame is None:
            print(f"Failed to read {sp}; skipping.")
            continue
        frame = cv2.resize(frame, frame_size)
        video_writer.write(frame)
    video_writer.release()
    print(f"Video saved to: {video_out}")

#########################################
# Main Function
#########################################

def main():
    trajectories = iterative_trajectory_generation()
    if trajectories is None or len(trajectories) == 0:
        print("No trajectories generated. Exiting.")
        return
    # For testing, take the first generated trajectory.
    traj = trajectories[0]
    print(f"Using first trajectory with {len(traj)} points for synthesis and video creation.")
    synthesize_and_create_video_for_trajectory(traj)

if __name__ == "__main__":
    main()


Available Demos: ['demo_0', 'demo_1', 'demo_2', 'demo_3', 'demo_4', 'demo_5']
Reference spherical coords: r=0.717, az=-1.481, el=0.304
Generated one trajectory with 284 points.
Using first trajectory with 284 points for synthesis and video creation.
Removing background from reference image...
Processed reference image saved to: temp_ref/eye_in_hand_rgb.png


text_encoder/model.safetensors not found
Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  2.52it/s]
text_encoder/model.safetensors not found
Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  2.97it/s]


Processing point 0: Δaz=-0.087, Δel=0.087, Δr=-0.000
input_image_path is: temp_ref/eye_in_hand_rgb.png


100%|██████████| 5/5 [00:00<00:00,  6.07it/s]


Saving output images.....
Processing point 1: Δaz=0.087, Δel=0.087, Δr=-0.001
input_image_path is: temp_ref/eye_in_hand_rgb.png


...

Saving output images.....
Processing point 282: Δaz=-0.256, Δel=-0.345, Δr=-0.717
input_image_path is: temp_ref/eye_in_hand_rgb.png


100%|██████████| 5/5 [00:00<00:00,  6.05it/s]


Saving output images.....
Processing point 283: Δaz=1.481, Δel=-0.304, Δr=-0.717
input_image_path is: temp_ref/eye_in_hand_rgb.png


100%|██████████| 5/5 [00:00<00:00,  6.06it/s]


Saving output images.....
Creating video from 284 frames...
Video saved to: trajectory_video.mp4
